## Calculating Transfer Function between two TE accelerometers with a labJack T7-Pro
Uses 9 analog inputs (AINs) to read the data at 1000 Hz.

Craig Lage - Sep 14, 2021

Relevant Documentation:
 
LJM Library:
    LJM Library Installer:
        https://labjack.com/support/software/installers/ljm
    LJM Users Guide:
        https://labjack.com/support/software/api/ljm
    Opening and Closing:
        https://labjack.com/support/software/api/ljm/function-reference/opening-and-closing
    Multiple Value Functions(such as eWriteNames):
        https://labjack.com/support/software/api/ljm/function-reference/multiple-value-functions
    Timing Functions(such as StartInterval):
        https://labjack.com/support/software/api/ljm/function-reference/timing-functions
 
T-Series and I/O:
    Modbus Map:
        https://labjack.com/support/software/api/modbus/modbus-map
    Analog Inputs:
        https://labjack.com/support/datasheets/t-series/ain

In [ ]:
import sys
import time, datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.fft import fft, fftfreq
from scipy.interpolate import splprep, splev

from labjack import ljm  # Needed pip install labjack-ljm

In [ ]:
# Open first found LabJack
handle = ljm.openS("T7", "ANY", "ANY")  # T7 device, Any connection, Any identifier

In [ ]:
info = ljm.getHandleInfo(handle)
print("Opened a LabJack with Device type: %i, Connection type: %i,\n"
      "Serial number: %i, IP address: %s, Port: %i,\nMax bytes per MB: %i" %
      (info[0], info[1], info[2], ljm.numberToIP(info[3]), info[4], info[5]))

deviceType = info[0]
print(deviceType)

In [ ]:
# Structures to hold the calibration data and connection information
class zeroOffset:
    def __init__(self, off_x, off_y, off_z):
        # This is the reading in Volts when accel = 0
        self.x = off_x
        self.y = off_y
        self.z = off_z
        
class gMult:
    def __init__(self, mult_x, mult_y, mult_z):
        # This is the conversion to acceleration in V/g
        self.x = mult_x
        self.y = mult_y
        self.z = mult_z
        
class AIN_name:
    def __init__(self, ain_x, ain_y, ain_z):
        # This is where the sensors are connected to the labJack
        self.x = ain_x
        self.y = ain_y
        self.z = ain_z

class calData:
    def __init__(self, serial="", ain_x="", ain_y="", ain_z="", off_x=0.0, off_y=0.0, off_z=0.0, mult_x=1.0, mult_y=1.0, mult_z=1.0):
        # The serial number is imprinted on the accelerometer
        self.serial = serial
        self.AIN_name = AIN_name(ain_x, ain_y, ain_z)
        self.zeroOffset = zeroOffset(off_x, off_y, off_z)
        self.gMult = gMult(mult_x, mult_y, mult_z)
        
calDict = {}
calDict["1"] = calData(serial="A395429", ain_x="AIN1", ain_y="AIN2", ain_z="AIN3", off_x=2.49017, off_y=2.44424, off_z=2.44589, mult_x=0.98959, mult_y=0.98572, mult_z=0.99946)
calDict["2"] = calData(serial="A395423", ain_x="AIN4", ain_y="AIN5", ain_z="AIN6", off_x=2.49874, off_y=2.49595, off_z=2.41423, mult_x=0.99740, mult_y=1.00142, mult_z=0.99595)
calDict["3"] = calData(serial="A395446", ain_x="AIN7", ain_y="AIN8", ain_z="AIN9", off_x=2.47830, off_y=2.48088, off_z=2.41385, mult_x=0.97957, mult_y=0.98699, mult_z=1.00376)

In [ ]:
# Create list of AIN names and initialize

# Ensure triggered stream is disabled.
ljm.eWriteName(handle, "STREAM_TRIGGER_INDEX", 0)
# Enabling internally-clocked stream.
ljm.eWriteName(handle, "STREAM_CLOCK_SOURCE", 0)

# All negative channels are single-ended, AIN0 and AIN1 ranges are
# +/-10 V, stream settling is 0 (default) and stream resolution index
# is 0 (default).

aRange = 10.0 # +/- 10.0 Volts
aSettle = 0 # 0 microsecond settling time
resIndex = 0
aNames = ["AIN_ALL_NEGATIVE_CH", "STREAM_SETTLING_US", "STREAM_RESOLUTION_INDEX"] # List of set-up parameters
aValues = [ljm.constants.GND, aSettle, resIndex] # List of set-up values

aScanListNames = [] # List of AIN names which will be read
offsets = []
gMults = []
for name in ["1", "2", "3"]:
    for axis in ["x", "y", "z"]:
        exec(f"aName = calDict['{name}'].AIN_name.{axis}")
        aScanListNames.append(aName)
        aNames.append(aName+"_RANGE")
        aValues.append(aRange)
        exec(f"off = calDict['{name}'].zeroOffset.{axis}")
        offsets.append(off)
        exec(f"gMult = calDict['{name}'].gMult.{axis}")
        gMults.append(gMult)

offsets = np.array(offsets)
gMults = np.array(gMults)
         
# Write the analog inputs' negative channels (when applicable), ranges,
# stream settling time and stream resolution configuration.
numFrames = len(aNames)
ljm.eWriteNames(handle, numFrames, aNames, aValues)


In [ ]:
def readStream(aScanListNames, handle=handle, scanRate=200, readTime=1.0, offsets=offsets, gMults=gMults):
    # This reads the accelerometers for a time readTime
    # and returns a Pandas timeSeries with the data
    # aScanListNames is the list of AIN ports (9 in total)
    # handle is the handle for talking to the labJack
    # scanRate is the read frequency in Hertz
    # readTime is the total time of read in seconds
    # calDict is the dictionary with the calibration data
    # The function returns a Pandas dataframe with the three 
    # accelerometers times three axes results
    
    numAddresses = len(aScanListNames)
    aScanList = ljm.namesToAddresses(numAddresses, aScanListNames)[0]
    scansPerRead = int(scanRate * readTime)
    try:
        # Configure and start stream
        scanRate = ljm.eStreamStart(handle, scansPerRead, numAddresses, aScanList, scanRate)
        start = datetime.datetime.now()
        # Stream the data
        ret = ljm.eStreamRead(handle)
        # Stop the stream
        ljm.eStreamStop(handle)
        aData = ret[0]
        # Reshape the data
        newData = np.resize(aData, (scansPerRead, numAddresses))
        # Convert to g
        accelData = (newData - offsets) / gMults
        # Create the timestamps
        end = start + datetime.timedelta(seconds = readTime)
        date_rng = pd.date_range(start=start, end=end, periods=scansPerRead)
        # Create the Pandas dataFrame
        df = pd.DataFrame(accelData, index=date_rng, 
                          columns=['a1x', 'a1y', 'a1z', 'a2x', 'a2y', 'a2z', 'a3x', 'a3y', 'a3z'])
    except ljm.LJMError:
        ljme = sys.exc_info()[1]
        print(ljme)
    except Exception:
        e = sys.exc_info()[1]
        print(e)
    # Return the dataframe
    return df

In [ ]:
time.sleep(2.0)
scanRate = 1000
readTime = 10.0
df = readStream(aScanListNames, handle=handle, scanRate=scanRate, readTime=readTime, offsets=offsets, gMults=gMults)

In [ ]:
df.to_pickle("/Users/cslage/Research/LSST/code/labJack/accel_data/Transfer_Test_6.pkl")

In [ ]:
df = pd.read_pickle("/Users/cslage/Research/LSST/code/labJack/accel_data/Transfer_Test_6.pkl")

In [ ]:
# Plot the data
plt.figure(figsize=(16,8))
plt.subplots_adjust(hspace=0.5)

for n, scanIndex in enumerate([[0, int(scanRate*readTime-1)], [450, 750], [500,600]]):

    sub_df = df[df.index[scanIndex[0]] : df.index[scanIndex[1]]]
    
    plt.subplot(2,3,n+1)
    ax1 = sub_df['a1z'].plot(label="a1z", color='red')
    ax1.set_title("Axis 1", fontsize=16)
    ax1.set_ylabel("Acceleration(g)")
    ax1.legend(loc='center left')

    plt.subplot(2,3,n+4)
    ax2 = sub_df['a2z'].plot(label="a2z", color='red')
    ax2.set_title("Axis 2", fontsize=16)
    ax2.set_ylabel("Acceleration(g)")
    ax2.legend(loc='center left')
    
    if n == 1: # save this data for the FFT
        a1z = np.array(sub_df['a1z'].tolist())
        a2z = np.array(sub_df['a2z'].tolist())

#plt.savefig("/Users/cslage/Research/LSST/code/labJack/accel_data/Transfer_Test_6_14Sep21.pdf")

In [ ]:
def calcTransferFunction(scanRate, drivenSeries, monitorSeries):
    # Calculates and plots a transfer function given a driven series of data
    # and a monitor series of data

    # Do the FFT
    N = len(drivenSeries)
    xAxis = fftfreq(N, 1 / scanRate)
    yf1 = fft(drivenSeries)
    yf2 = fft(monitorSeries)
    
    # Now plot it

    rmsRatio = np.std(monitorSeries) / np.std(drivenSeries)
    fftPeakRatio = np.max(np.abs(yf2[50:-1])) / np.max(np.abs(yf1[50:-1]))
    #fftPeakRatio = np.max(np.abs(yf2)) / np.max(np.abs(yf1))

    plt.figure(figsize=(16,8))
    plt.subplot(1,2,1)
    plt.subplots_adjust(hspace=0.5)
    plt.plot(xAxis, np.abs(yf1), color='red', marker = 'x', label='A1Z - Driver')
    plt.plot(xAxis, np.abs(yf2), color='green', marker = 'x', label='A2Z - Monitor')
    plt.title("FFT of accelerometer force transfer", fontsize=24)
    plt.xlabel("Frequency (Hz)",fontsize=16)
    #plt.text(20, 20, "RMS_Ratio = %.3f"%rmsRatio, fontsize=16)
    #plt.text(20, 17, "FFT Peak Ratio = %.3f"%fftPeakRatio, fontsize=16)
    plt.ylim(0,2.0)
    plt.xlim(10,400)
    plt.legend(loc = 'upper left', fontsize=12)

    # Calculate the transfer function vs frequency and a spline fit and plot them
    
    transferFunction = []
    frequency = []
    for i, x in enumerate(xAxis):
        if x > 9.9 and x < 400.0:
            frequency.append(x)
            transferFunction.append(np.abs(yf2[i]) / np.abs(yf1[i]))
            nKnots = 20
    tck, u = splprep([frequency, transferFunction],t=nKnots,k=3,nest=-1)
    fFit, tfFit = splev(u,tck)
    plt.subplot(1,2,2)
    plt.title("Transfer Function", fontsize=24)
    plt.plot(frequency, transferFunction, marker='x', color='blue', label = 'Raw Transfer Function')
    plt.plot(fFit, tfFit, lw=5.0, color='red', label = 'Spline Fit')
    plt.xlabel("Frequency(Hz)", fontsize=16)
    plt.ylabel("Acceleration Ratio", fontsize=16)
    plt.legend(fontsize=12)
    plt.savefig("/Users/cslage/Research/LSST/code/labJack/accel_data/Transfer_Test_6_Plot_14Sep21.pdf")
    
    return (frequency, transferFunction)

In [ ]:
(frequency, transferFunction) = calcTransferFunction(scanRate, a1z, a2z)

In [ ]:
# Close handles
ljm.close(handle)